In [5]:
import sys
from datetime import datetime, timedelta
import numpy as np
import tqdm
import pandas as pd
import oandapy

sys.path.append('/home/toshio/project/fx')
from config import token


class CandleFetcher(object):

    name = "NZDJPY_M15"
    currency_pair = "NZD_JPY"
    unit = "NZD_JPY"
    granularity = "M15"
    oanda = oandapy.API(environment="practice", access_token=token)

    def fetch(self, start_t, end_t):
        save_name = "../intermediate_data/{}_{}_{}.pkl".format(
            self.name,
            start_t.strftime('%Y%m%d@%H%M%S'),
            end_t.strftime('%Y%m%d@%H%M%S'))
        dates = []
        minutes_cls = []
        minutes_max = []
        minutes_min = []
        minutes_opn = []
        nth = 0
        mod = (end_t - start_t)
        dmin = int(min(60 * 27 * 3 , max(mod.days * 24 * 60 + mod.seconds // 60, 1))) #M15以上のときは、60*27*3*15
        mod = timedelta(minutes=dmin)
        bar = tqdm.tqdm(range(int(np.ceil((end_t - start_t) / mod))))
        
        while start_t + mod * nth < end_t:
            start = datetime.strftime(start_t + mod * nth, '%Y-%m-%dT%H:%M:%SZ')
            end = datetime.strftime(min(start_t + mod * (nth + 1), end_t), '%Y-%m-%dT%H:%M:%SZ')
            response = self.oanda.get_history(
                instrument=self.currency_pair,
                granularity=self.granularity,
                start=start,
                end=end,
                candleFormat="midpoint",
            )

            for res in sorted(response["candles"], key=lambda x: x["time"]):
                md = datetime.strptime(res["time"][:-8], '%Y-%m-%dT%H:%M:%S')
                dates.append(md)
                minutes_opn.append(res["openMid"])
                minutes_cls.append(res["closeMid"])
                minutes_min.append(res["lowMid"])
                minutes_max.append(res["highMid"])

            data = {
                "date": dates,
                "open": minutes_opn,
                "close": minutes_cls,
                "high": minutes_max,
                "low": minutes_min,
            }
            bar.set_description("Get {} ~ {}".format(start, end))
            bar.update(1)
            nth += 1

        df = pd.DataFrame(data, columns=["date", "open", "close", "high", "low"])
        df.to_pickle(save_name)
        print(df.head(5))


if __name__ == "__main__":
    fet = CandleFetcher()
    fet.fetch(datetime(2010, 1, 1), datetime(2018, 8, 23))





  0%|          | 0/936 [00:00<?, ?it/s]



Get 2010-01-01T00:00:00Z ~ 2010-01-04T09:00:00Z:   0%|          | 0/936 [00:01<?, ?it/s]



Get 2010-01-01T00:00:00Z ~ 2010-01-04T09:00:00Z:   0%|          | 1/936 [00:01<27:39,  1.77s/it]



Get 2010-01-04T09:00:00Z ~ 2010-01-07T18:00:00Z:   0%|          | 1/936 [00:05<27:39,  1.77s/it]



Get 2010-01-04T09:00:00Z ~ 2010-01-07T18:00:00Z:   0%|          | 2/936 [00:05<35:06,  2.26s/it]



Get 2010-01-07T18:00:00Z ~ 2010-01-11T03:00:00Z:   0%|          | 2/936 [00:05<35:06,  2.26s/it]



Get 2010-01-07T18:00:00Z ~ 2010-01-11T03:00:00Z:   0%|          | 3/936 [00:05<27:34,  1.77s/it]



Get 2010-01-11T03:00:00Z ~ 2010-01-14T12:00:00Z:   0%|          | 3/936 [00:06<27:34,  1.77s/it]



Get 2010-01-11T03:00:00Z ~ 2010-01-14T12:00:00Z:   0%|          | 4/936 [00:06<23:43,  1.53s/it]



Get 2010-01-14T12:00:00Z ~ 2010-01-17T21:00:00Z:   0%|          | 4/936 [00:07<23:43,  1.53s/it]



Get 2010-01-14T12:00:00Z ~ 2010-01-17T21:00:00Z:   1%|       

Get 2010-05-02T12:00:00Z ~ 2010-05-05T21:00:00Z:   4%|▍         | 36/936 [01:10<27:26,  1.83s/it]



Get 2010-05-02T12:00:00Z ~ 2010-05-05T21:00:00Z:   4%|▍         | 37/936 [01:10<23:18,  1.56s/it]



Get 2010-05-05T21:00:00Z ~ 2010-05-09T06:00:00Z:   4%|▍         | 37/936 [01:11<23:18,  1.56s/it]



Get 2010-05-05T21:00:00Z ~ 2010-05-09T06:00:00Z:   4%|▍         | 38/936 [01:11<20:16,  1.35s/it]



Get 2010-05-09T06:00:00Z ~ 2010-05-12T15:00:00Z:   4%|▍         | 38/936 [01:16<20:16,  1.35s/it]



Get 2010-05-09T06:00:00Z ~ 2010-05-12T15:00:00Z:   4%|▍         | 39/936 [01:16<37:39,  2.52s/it]



Get 2010-05-12T15:00:00Z ~ 2010-05-16T00:00:00Z:   4%|▍         | 39/936 [01:17<37:39,  2.52s/it]



Get 2010-05-12T15:00:00Z ~ 2010-05-16T00:00:00Z:   4%|▍         | 40/936 [01:17<31:54,  2.14s/it]



Get 2010-05-16T00:00:00Z ~ 2010-05-19T09:00:00Z:   4%|▍         | 40/936 [01:18<31:54,  2.14s/it]



Get 2010-05-16T00:00:00Z ~ 2010-05-19T09:00:00Z:   4%|▍         | 41/936 [01:18<26:47,  1.8

Get 2010-09-01T00:00:00Z ~ 2010-09-04T09:00:00Z:   8%|▊         | 72/936 [02:14<12:46,  1.13it/s]



Get 2010-09-01T00:00:00Z ~ 2010-09-04T09:00:00Z:   8%|▊         | 73/936 [02:14<11:45,  1.22it/s]



Get 2010-09-04T09:00:00Z ~ 2010-09-07T18:00:00Z:   8%|▊         | 73/936 [02:17<11:45,  1.22it/s]



Get 2010-09-04T09:00:00Z ~ 2010-09-07T18:00:00Z:   8%|▊         | 74/936 [02:17<19:04,  1.33s/it]



Get 2010-09-07T18:00:00Z ~ 2010-09-11T03:00:00Z:   8%|▊         | 74/936 [02:18<19:04,  1.33s/it]



Get 2010-09-07T18:00:00Z ~ 2010-09-11T03:00:00Z:   8%|▊         | 75/936 [02:18<20:01,  1.40s/it]



Get 2010-09-11T03:00:00Z ~ 2010-09-14T12:00:00Z:   8%|▊         | 75/936 [02:19<20:01,  1.40s/it]



Get 2010-09-11T03:00:00Z ~ 2010-09-14T12:00:00Z:   8%|▊         | 76/936 [02:19<16:47,  1.17s/it]



Get 2010-09-14T12:00:00Z ~ 2010-09-17T21:00:00Z:   8%|▊         | 76/936 [02:20<16:47,  1.17s/it]



Get 2010-09-14T12:00:00Z ~ 2010-09-17T21:00:00Z:   8%|▊         | 77/936 [02:20<15:31,  1.0

Get 2010-12-31T12:00:00Z ~ 2011-01-03T21:00:00Z:  12%|█▏        | 108/936 [03:00<14:58,  1.08s/it]



Get 2010-12-31T12:00:00Z ~ 2011-01-03T21:00:00Z:  12%|█▏        | 109/936 [03:00<12:59,  1.06it/s]



Get 2011-01-03T21:00:00Z ~ 2011-01-07T06:00:00Z:  12%|█▏        | 109/936 [03:01<12:59,  1.06it/s]



Get 2011-01-03T21:00:00Z ~ 2011-01-07T06:00:00Z:  12%|█▏        | 110/936 [03:01<12:59,  1.06it/s]



Get 2011-01-07T06:00:00Z ~ 2011-01-10T15:00:00Z:  12%|█▏        | 110/936 [03:01<12:59,  1.06it/s]



Get 2011-01-07T06:00:00Z ~ 2011-01-10T15:00:00Z:  12%|█▏        | 111/936 [03:01<10:21,  1.33it/s]



Get 2011-01-10T15:00:00Z ~ 2011-01-14T00:00:00Z:  12%|█▏        | 111/936 [03:09<10:21,  1.33it/s]



Get 2011-01-10T15:00:00Z ~ 2011-01-14T00:00:00Z:  12%|█▏        | 112/936 [03:09<37:53,  2.76s/it]



Get 2011-01-14T00:00:00Z ~ 2011-01-17T09:00:00Z:  12%|█▏        | 112/936 [03:10<37:53,  2.76s/it]



Get 2011-01-14T00:00:00Z ~ 2011-01-17T09:00:00Z:  12%|█▏        | 113/936 [03:10<3

Get 2011-04-28T15:00:00Z ~ 2011-05-02T00:00:00Z:  15%|█▌        | 144/936 [03:49<14:26,  1.09s/it]



Get 2011-05-02T00:00:00Z ~ 2011-05-05T09:00:00Z:  15%|█▌        | 144/936 [03:50<14:26,  1.09s/it]



Get 2011-05-02T00:00:00Z ~ 2011-05-05T09:00:00Z:  15%|█▌        | 145/936 [03:50<13:47,  1.05s/it]



Get 2011-05-05T09:00:00Z ~ 2011-05-08T18:00:00Z:  15%|█▌        | 145/936 [03:51<13:47,  1.05s/it]



Get 2011-05-05T09:00:00Z ~ 2011-05-08T18:00:00Z:  16%|█▌        | 146/936 [03:51<11:53,  1.11it/s]



Get 2011-05-08T18:00:00Z ~ 2011-05-12T03:00:00Z:  16%|█▌        | 146/936 [03:54<11:53,  1.11it/s]



Get 2011-05-08T18:00:00Z ~ 2011-05-12T03:00:00Z:  16%|█▌        | 147/936 [03:54<22:52,  1.74s/it]



Get 2011-05-12T03:00:00Z ~ 2011-05-15T12:00:00Z:  16%|█▌        | 147/936 [03:55<22:52,  1.74s/it]



Get 2011-05-12T03:00:00Z ~ 2011-05-15T12:00:00Z:  16%|█▌        | 148/936 [03:55<19:38,  1.50s/it]



Get 2011-05-15T12:00:00Z ~ 2011-05-18T21:00:00Z:  16%|█▌        | 148/936 [03:56<1

...

Get 2017-07-23T18:00:00Z ~ 2017-07-27T03:00:00Z:  87%|████████▋ | 818/936 [19:22<01:41,  1.17it/s]



Get 2017-07-23T18:00:00Z ~ 2017-07-27T03:00:00Z:  88%|████████▊ | 819/936 [19:22<01:42,  1.14it/s]



Get 2017-07-27T03:00:00Z ~ 2017-07-30T12:00:00Z:  88%|████████▊ | 819/936 [19:22<01:42,  1.14it/s]



Get 2017-07-27T03:00:00Z ~ 2017-07-30T12:00:00Z:  88%|████████▊ | 820/936 [19:22<01:33,  1.24it/s]



Get 2017-07-30T12:00:00Z ~ 2017-08-02T21:00:00Z:  88%|████████▊ | 820/936 [19:23<01:33,  1.24it/s]



Get 2017-07-30T12:00:00Z ~ 2017-08-02T21:00:00Z:  88%|████████▊ | 821/936 [19:23<01:32,  1.24it/s]



Get 2017-08-02T21:00:00Z ~ 2017-08-06T06:00:00Z:  88%|████████▊ | 821/936 [19:24<01:32,  1.24it/s]



Get 2017-08-02T21:00:00Z ~ 2017-08-06T06:00:00Z:  88%|████████▊ | 822/936 [19:24<01:22,  1.39it/s]



Get 2017-08-06T06:00:00Z ~ 2017-08-09T15:00:00Z:  88%|████████▊ | 822/936 [19:24<01:22,  1.39it/s]



Get 2017-08-06T06:00:00Z ~ 2017-08-09T15:00:00Z:  88%|████████▊ | 823/936 [19:24<0

Get 2017-11-18T21:00:00Z ~ 2017-11-22T06:00:00Z:  91%|█████████ | 854/936 [19:48<01:32,  1.13s/it]



Get 2017-11-22T06:00:00Z ~ 2017-11-25T15:00:00Z:  91%|█████████ | 854/936 [19:48<01:32,  1.13s/it]



Get 2017-11-22T06:00:00Z ~ 2017-11-25T15:00:00Z:  91%|█████████▏| 855/936 [19:48<01:25,  1.06s/it]



Get 2017-11-25T15:00:00Z ~ 2017-11-29T00:00:00Z:  91%|█████████▏| 855/936 [19:49<01:25,  1.06s/it]



Get 2017-11-25T15:00:00Z ~ 2017-11-29T00:00:00Z:  91%|█████████▏| 856/936 [19:49<01:15,  1.07it/s]



Get 2017-11-29T00:00:00Z ~ 2017-12-02T09:00:00Z:  91%|█████████▏| 856/936 [19:52<01:15,  1.07it/s]



Get 2017-11-29T00:00:00Z ~ 2017-12-02T09:00:00Z:  92%|█████████▏| 857/936 [19:52<01:49,  1.39s/it]



Get 2017-12-02T09:00:00Z ~ 2017-12-05T18:00:00Z:  92%|█████████▏| 857/936 [19:52<01:49,  1.39s/it]



Get 2017-12-02T09:00:00Z ~ 2017-12-05T18:00:00Z:  92%|█████████▏| 858/936 [19:52<01:36,  1.23s/it]



Get 2017-12-05T18:00:00Z ~ 2017-12-09T03:00:00Z:  92%|█████████▏| 858/936 [19:54<0

Get 2018-03-20T09:00:00Z ~ 2018-03-23T18:00:00Z:  95%|█████████▍| 889/936 [20:28<00:47,  1.02s/it]



Get 2018-03-20T09:00:00Z ~ 2018-03-23T18:00:00Z:  95%|█████████▌| 890/936 [20:28<00:46,  1.01s/it]



Get 2018-03-23T18:00:00Z ~ 2018-03-27T03:00:00Z:  95%|█████████▌| 890/936 [20:28<00:46,  1.01s/it]



Get 2018-03-23T18:00:00Z ~ 2018-03-27T03:00:00Z:  95%|█████████▌| 891/936 [20:28<00:35,  1.25it/s]



Get 2018-03-27T03:00:00Z ~ 2018-03-30T12:00:00Z:  95%|█████████▌| 891/936 [20:29<00:35,  1.25it/s]



Get 2018-03-27T03:00:00Z ~ 2018-03-30T12:00:00Z:  95%|█████████▌| 892/936 [20:29<00:37,  1.16it/s]



Get 2018-03-30T12:00:00Z ~ 2018-04-02T21:00:00Z:  95%|█████████▌| 892/936 [20:29<00:37,  1.16it/s]



Get 2018-03-30T12:00:00Z ~ 2018-04-02T21:00:00Z:  95%|█████████▌| 893/936 [20:29<00:29,  1.44it/s]



Get 2018-04-02T21:00:00Z ~ 2018-04-06T06:00:00Z:  95%|█████████▌| 893/936 [20:30<00:29,  1.44it/s]



Get 2018-04-02T21:00:00Z ~ 2018-04-06T06:00:00Z:  96%|█████████▌| 894/936 [20:30<0

Get 2018-07-16T12:00:00Z ~ 2018-07-19T21:00:00Z:  99%|█████████▉| 925/936 [21:06<00:16,  1.50s/it]



Get 2018-07-19T21:00:00Z ~ 2018-07-23T06:00:00Z:  99%|█████████▉| 925/936 [21:06<00:16,  1.50s/it]



Get 2018-07-19T21:00:00Z ~ 2018-07-23T06:00:00Z:  99%|█████████▉| 926/936 [21:06<00:12,  1.23s/it]



Get 2018-07-23T06:00:00Z ~ 2018-07-26T15:00:00Z:  99%|█████████▉| 926/936 [21:07<00:12,  1.23s/it]



Get 2018-07-23T06:00:00Z ~ 2018-07-26T15:00:00Z:  99%|█████████▉| 927/936 [21:07<00:10,  1.17s/it]



Get 2018-07-26T15:00:00Z ~ 2018-07-30T00:00:00Z:  99%|█████████▉| 927/936 [21:08<00:10,  1.17s/it]



Get 2018-07-26T15:00:00Z ~ 2018-07-30T00:00:00Z:  99%|█████████▉| 928/936 [21:08<00:07,  1.09it/s]



Get 2018-07-30T00:00:00Z ~ 2018-08-02T09:00:00Z:  99%|█████████▉| 928/936 [21:08<00:07,  1.09it/s]



Get 2018-07-30T00:00:00Z ~ 2018-08-02T09:00:00Z:  99%|█████████▉| 929/936 [21:08<00:05,  1.17it/s]



Get 2018-08-02T09:00:00Z ~ 2018-08-05T18:00:00Z:  99%|█████████▉| 929/936 [21:09<0

                 date    open   close    high     low
0 2010-01-03 18:00:00  67.261  67.223  67.261  67.223
1 2010-01-03 18:15:00  67.280  67.264  67.280  67.179
2 2010-01-03 18:30:00  67.259  67.283  67.287  67.255
3 2010-01-03 18:45:00  67.278  67.286  67.289  67.269
4 2010-01-03 19:00:00  67.289  67.231  67.289  67.141
